In [55]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv('data/properatti.csv')
data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.00,"-34.6618237,-58.5088387",-34.66,...,40.00,1127.27,1550.00,nan,nan,nan,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.00,"-34.9038831,-57.9643295",-34.90,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.00,"-34.6522615,-58.5229825",-34.65,...,55.00,1309.09,1309.09,nan,nan,nan,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.00,"-34.6477969,-58.5164244",-34.65,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.00,"-38.0026256,-57.5494468",-38.00,...,35.00,1828.57,1828.57,nan,nan,nan,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [56]:
''' Vemos las columnas '''

data.loc[10,:]

Unnamed: 0                                                                   10
operation                                                                  sell
property_type                                                             house
place_name                                                              Córdoba
place_with_parent_names                             |Argentina|Córdoba|Córdoba|
country_name                                                          Argentina
state_name                                                              Córdoba
geonames_id                                                          3860259.00
lat-lon                                                 -31.4200833,-64.1887761
lat                                                                      -31.42
lon                                                                      -64.19
price                                                                  70000.00
currency                                

In [57]:
''' revisamos en que moneda se encuentran expresadas las propiedades'''
data.currency.value_counts()

USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64

In [58]:
''' de las columnas precios vamos a analizar price_aprox_usd para tomarla como referencia'''
''' vemos los nulos '''

data['price_aprox_usd'].isna().sum()

20410

In [59]:
''' Exploramos las columnas de los precios para ver si hay datos no nulos'''

pricenull = data.loc[data['price_aprox_usd'].isna()].loc[:,['price','price_aprox_local_currency','price_usd_per_m2','price_per_m2']]

pricenull.isna().sum()



price                         20410
price_aprox_local_currency    20410
price_usd_per_m2              20410
price_per_m2                  20410
dtype: int64

In [60]:
''' quitar los nulos precios '''

data = data.drop(data[data['price_aprox_usd'].isna()].index)
cant_nulls = data.price_aprox_usd.isnull().sum()
print(cant_nulls)

0


In [61]:
#Veo cuántos nulos tiene la columna place_name
data.place_name.isnull().sum()

23

In [62]:
''' completar los places names '''
data.loc[data.place_name.isnull(),'place_name'] = data.loc[data.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]

'''place names nulls despues de completar'''

data.place_name.isnull().sum()

0

In [63]:
#Relleno manualmente un dato que está suelto
mask=(data.place_name=='coordenadas 34.255511')
data.loc[mask,'place_name'] = 'Rincón de Milberg'


In [64]:
''' columna ambientes'''
data.rooms.isna().sum()

61231

In [65]:
#Relleno con patron regex segun la palabra ambiente
patron = '((?P<numero>\d|mono|dos|tres|cuatro|cinco|seis)\s*(?P<ambiente>amb))'
patron_regex = re.compile(patron,flags = re.IGNORECASE)

def convertRoom(row):
    try:
        if (row['description'] and isinstance(row['description'],str)):
            resultado = patron_regex.search(row['description'])
        if ((resultado is None) and row['title'] and isinstance(row['title'],str)):
            resultado = patron_regex.search(row['title'])
        return resultado.group('numero')
    except:
        return np.nan
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(convertRoom,axis=1)

patron_ambientes = re.compile("(?P<Ambientes>\d+)(?P<tipo>amb|\samb)",flags = re.IGNORECASE)
patron_habitaciones = re.compile("(?P<Habitaciones>\d+)(?P<tipo>hab|\shab|dorm|\sdorm)",flags = re.IGNORECASE)

def extractRooms(row):
    ambientes_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        ambientes_match = patron_ambientes.search(row['description'])
    if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
        ambientes_match = patron_ambientes.search(row['title'])
    if (ambientes_match is not None):
        return int(ambientes_match.groups()[0]) 
    else: 
        if (row['description'] and isinstance(row['description'],str)):
            ambientes_match = patron_habitaciones.search(row['description'])
        if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
            ambientes_match = patron_habitaciones.search(row['title'])
        if (ambientes_match is not None):
            return int(ambientes_match.groups()[0])+1
        return None
        
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(extractRooms,axis=1)

''' rooms despues de limpiar'''

data.rooms.isnull().sum()


19604

In [66]:
''' nulos en superficie'''

data.surface_total_in_m2.isnull().sum()

31944

In [67]:

patron_meters = re.compile("(?P<Metros>\d+)(?P<tipo>metro|\smetro|mts|\smts|m2|\sm2)",flags = re.IGNORECASE)


def extractMeters(row):
    meters_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        meters_match = patron_meters.search(row['description'])
    if ((meters_match is None) and row['title'] and isinstance(row['title'],str)):
        meters_match = patron_meters.search(row['title'])
    if (meters_match is not None):
        return meters_match.groups()[0]
    return None

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].apply(extractMeters,axis=1)

data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()].apply(lambda x: x['surface_covered_in_m2'],axis=1)




In [68]:
data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].surface_total_in_m2.fillna(value=np.nan)
data.loc[:,'surface_total_in_m2'] = data.loc[:,'surface_total_in_m2'].astype("float")

''' cuantos quedaron'''

data.surface_total_in_m2.isnull().sum()

5892

Generamos columnas para categorizar los departamentos, al frente, contrafrente, lateral


In [69]:
''' columna disposicion del dpto'''

patron_disposicion = re.compile("(?P<tipo>frente|lateral|contrafrente)",flags = re.IGNORECASE)

def disposicion(row):
    disposicion_match = None
    if ( row['description'] and isinstance(row['description'],str) ): 
        disposicion_match = patron_disposicion.search(row['description'])
    if ((patron_disposicion is None) and row['title'] and isinstance(row['title'],str)):
        disposicion_match = patron_disposicion.search(row['title'])
    if (disposicion_match is not None):
        return disposicion_match.groups()[0].lower()
    return None  

data['disposicion'] =  data.apply(lambda x:disposicion(x),axis = 1)
data['disposicion'].value_counts()

frente          28536
contrafrente     5262
lateral          1811
Name: disposicion, dtype: int64

In [70]:
''' cuantos departamentos tienen el piso en null '''
len(data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].index)

53700

In [71]:
patron_floor = re.compile("(?P<piso0>planta baja|PB)|(?P<piso1>1er piso|primer piso|piso 1|1º piso|piso 1º)|(?P<piso2>2do piso|segundo piso|piso 2|2º piso|piso 2º)|(?P<piso3>3er piso|tercer piso|piso 3|3º piso|piso 3º)|(?P<piso4>4to piso|cuarto piso|piso 4|4º piso|piso 4º)|(?P<piso5>5to piso|quinto piso|piso 5|5º piso|piso 5º)|(?P<piso6>6to piso|sexto piso|piso 6|6º piso|piso 6º)|(?P<piso7>7to piso|séptimo piso|piso 7|7º piso|piso 7º)|(?P<piso8>8vo piso|octavo piso|piso 8|8º piso|piso 8º)|(?P<piso9>9no piso|noveno piso|piso 9|9º piso|piso 9º)|(?P<piso10>10mo piso|décimo piso|piso 10|10º piso|piso 10º)|(?P<piso11>11vo piso|onceavo piso|piso 11|11º piso|piso 11º)|(?P<piso12>12vo piso|doceavo piso|piso 12|12º piso|piso 12º)|(?P<piso13>13vo piso|treceavo piso|piso 13|13º piso|piso 13º)|(?P<piso14>14vo piso|cartoceavo piso|piso 14|14º piso|piso 14º)|(?P<piso15>15vo piso|quinceavo piso|piso 15|15º piso|piso 15º)",flags = re.IGNORECASE)

def extractFloor(row):
    match = None
    if (row['description'] and isinstance(row['description'],str) ):
        match = patron_floor.match(row['description'])
    if ( (match is None) and row['title'] and isinstance(row['title'],str) ):
        match = patron_floor.match(row['title'])
    if (match is not None):    
        return int(match.lastgroup.replace('piso',''))               
    return row['floor']

data.loc[(data.property_type == 'apartment') & (data.floor.isnull()),'floor'] = data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].apply(lambda x: extractFloor(x),axis =1)

''' cuantos departamentos tienen el piso en null '''
len(data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].index)

53413

In [72]:
''' columna a estrenar '''
patron_a_estrenar = re.compile("(?P<tipo>estrenar)",flags = re.IGNORECASE)

def aestrenar(row):
    if ( row['description'] and isinstance(row['description'],str) ):
        if (patron_a_estrenar.match(row['description'])):
                return True
    if ( row['title'] and isinstance(row['title'],str) ):
        if (patron_a_estrenar.match(row['title'])):
            return True
    return False

data['aAstrenar'] = data.apply(lambda x:aestrenar(x),axis = 1);



In [73]:
data['aAstrenar'].value_counts()

False    100801
True          9
Name: aAstrenar, dtype: int64

In [74]:
''' nos vamos a quedar con capital federal'''

data = data.loc[(data.state_name == 'Capital Federal')]
data.place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo Soho', 'Palermo',
       'Flores', 'Boedo', 'Las Cañitas', 'Puerto Madero', 'Balvanera',
       'Caballito', 'Nuñez', 'San Telmo', 'Almagro', 'Capital Federal',
       'Colegiales', 'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta',
       'Congreso', 'Villa Crespo', 'Chacarita', 'Constitución',
       'Villa Urquiza', 'Palermo Hollywood', 'Saavedra', 'Monserrat',
       'Pompeya', 'Parque Chas', 'Paternal', 'Agronomía',
       'Villa Pueyrredón', 'Coghlan', 'Parque Centenario', 'Villa Luro',
       'Villa Devoto', 'Boca', 'Parque Avellaneda', 'San Cristobal',
       'Abasto', 'Versalles', 'Villa del Parque', 'Monte Castro',
       'Retiro', 'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Palermo Chico', 'Centro / Microcentro', 'Tribunales', 'Once',
       'Parque Chacabuco', 'Velez Sarsfield', 'Catalinas',
       'Villa General Mitre', 'Palermo Viejo', 'Villa Lugano',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Re

In [75]:
#1. Copio los barrios y los Palermos los convierto a Palermo
data['place_name'] = data.place_name.apply(lambda x: "Palermo" if "Palermo" in x else x)

In [76]:
data.place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo', 'Flores', 'Boedo',
       'Las Cañitas', 'Puerto Madero', 'Balvanera', 'Caballito', 'Nuñez',
       'San Telmo', 'Almagro', 'Capital Federal', 'Colegiales',
       'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta', 'Congreso',
       'Villa Crespo', 'Chacarita', 'Constitución', 'Villa Urquiza',
       'Saavedra', 'Monserrat', 'Pompeya', 'Parque Chas', 'Paternal',
       'Agronomía', 'Villa Pueyrredón', 'Coghlan', 'Parque Centenario',
       'Villa Luro', 'Villa Devoto', 'Boca', 'Parque Avellaneda',
       'San Cristobal', 'Abasto', 'Versalles', 'Villa del Parque',
       'Monte Castro', 'Retiro', 'Parque Patricios', 'San Nicolás',
       'Villa Santa Rita', 'Centro / Microcentro', 'Tribunales', 'Once',
       'Parque Chacabuco', 'Velez Sarsfield', 'Catalinas',
       'Villa General Mitre', 'Villa Lugano', 'Villa Ortuzar',
       'Villa Soldati', 'Villa Real', 'Villa Riachuelo'], dtype=object)

In [77]:
''' empezamos a buscar outliers'''

pd.options.display.float_format = '{:.2f}'.format

data.price_per_m2.describe()

count     27324.00
mean       6108.16
std       23654.00
min           3.21
25%        2190.92
50%        2777.42
75%        3590.00
max     2600000.00
Name: price_per_m2, dtype: float64

In [89]:

def price_m2(row):
    if (row['surface_total_in_m2'] > 0):
        return row['price_aprox_usd'] / row['surface_total_in_m2']
    return row['price_per_m2']

data.loc[data.price_per_m2 > 5000,'price_per_m2'] = data.loc[data.price_per_m2 > 5000].apply(lambda x:  price_m2(x),axis=1)
data.price_per_m2.describe()

count     27324.00
mean       3726.80
std       13646.64
min           0.47
25%        2152.11
50%        2700.00
75%        3391.98
max     1050000.00
Name: price_per_m2, dtype: float64

In [90]:
data = data.drop(data.loc[data.price_per_m2 > 10000].index)
data.price_per_m2.describe()

count   26519.00
mean     2845.73
std      1137.52
min         0.47
25%      2131.80
50%      2666.67
75%      3306.29
max     10000.00
Name: price_per_m2, dtype: float64